
# Estimación de Poses con IA utilizando Python y MediaPipe
+ Proyecto de Seguimiento de Gimnasio

### Importamos las dependencias

In [1]:
import cv2
import mediapipe as mp# esto nos da las soluciones
import numpy as np
mp_dibujo= mp.solutions.drawing_utils
mp_pose= mp.solutions.pose

### Creamos el feed para nuestra webcam

In [6]:

#CODIGO BASE QUE REUTILIZAREMOS MAS TARDE 
#feed de video
cap= cv2.VideoCapture(0)# este numero representa mi camara
while cap.isOpened():
  ret,frame=cap.read()# esto nos guarda el feed de nuestra webcam en estas variables 
  cv2.imshow('Mediapipe Feed', frame)# aca mostramos la imagen de la webcam 

  if cv2.waitKey(10) & 0xFF == ord('q'):# esto chequea si le damos a "q" o cerramos la ventana y rompe el loop 
    break
cap.release()# liveramos nuestraaa webcam 
cv2.destroyAllWindows()#cierra la ventana 


### Determinar las Uniones que vamos a utilizar 

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px">

In [ ]:
# esta es la forma de extraer y mapear los puntos de referencia 'landarks'
#vamos a usarlo en el codigo final
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility

0.9999691843986511

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.65621895
y: 0.8706873
z: -0.08311059
visibility: 0.97787

### Calcular los Angulos:

Creamos una función que calcule ángulos y la utilizaremos en el hombro, codo y muñeca para calcular el ángulo del brazo contraído.

In [3]:
def calcular_angulo(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [23]:
#obtenemos los valores x e y:
Hombro=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
Codo=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
Muneca= [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

Hombro,Codo,Muneca

([0.5285757184028625, 0.7837801575660706],
 [0.6449593305587769, 0.8617578744888306],
 [0.7381910681724548, 0.8755338191986084])

In [24]:
calcular_angulo(Hombro,Codo,Muneca)

154.5828981324397

In [6]:
#UTILIZAMOS EL CODIGO BASE ANTERIOR
#IMPORTANTE PARA CORRER ESTE CODIGO PRIMERO DEBES IMPORTAR Y CREAR LA FUNCION 
cap = cv2.VideoCapture(0)

#Contador de curls
counter=0
stage=None

#Configuramos la instancia de mediapipe
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5)as pose:#acedemos al modelo de estimacion Pose y le especificamos las metricasa de confianza de la camaara y acedemos a la variable como pose
  while cap.isOpened():
      ret, frame = cap.read()
      # recololear la imagen porque cuando la pasemos a mediapipe la queremos en RGB por defaul esta en BGR
      image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)# tomamos el feed de laa camara y lo reorgaanizamos
      image.flags.writeable=False#ahorramos memoria
      # realizar la deteccion 
      results=pose.process(image)# guardamos la detecciones en la variable resultados 
      image.flags.writeable=True
      # recoloreamos a BGR
      image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)# volvemos a a recolorear a BGR parA renderizarrla en opencv que la quiere en BGR
      # Extraer  puntos de referencia
      try:
          landmarks = results.pose_landmarks.landmark
          
          #Obtener las coordenadas
          Hombro=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
          Codo=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
          Muneca= [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
          #Calcular el Angulo CON 3 PUNTOS DE REFERENCIA 
          angulo=calcular_angulo(Hombro,Codo,Muneca)
          #Visualizar el angulo 
          cv2.putText(image,str(angulo),
                      tuple(np.multiply(Codo,[640,480]).astype(int)),# pasamos la imagen de nuestra camara, el angulo y determinamos donde estmos posicionados teniendo en cuenta la camara 
                      cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
      
      #Logica del contador de Curls 
          if angulo > 160:
              stage = "down"
          if angulo < 30 and stage =='down':
              stage="up"
              counter +=1
              print(counter)
      
      except:
          pass
      #Renderizar el contador 
      #Configurar caja 
      cv2.rectangle(image, (0,0), (260,73), (245,117,16), -1)
      #Data de las Rep
      cv2.putText(image, 'REPS', (15,12), 
                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
      cv2.putText(image, str(counter), 
                  (10,60), 
                  cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
      # Posicion del brazo 
      cv2.putText(image, 'ESTADO', (65,12), 
                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
      cv2.putText(image, stage, 
                  (100,60), 
                  cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
      
      # Renderizar las Detecciones
      mp_dibujo.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                               mp_dibujo.DrawingSpec(color=(245,117,66), thickness=2,circle_radius=2),
                               mp_dibujo.DrawingSpec(color=(245,66,230), thickness=2,circle_radius=2)
                               )#Con esto dibujamos nuestras detecciones a nuestra imagen y personalizamos 
      
      cv2.imshow('Mediapipe Feed', image)
      
      if cv2.waitKey(10) & 0xFF == ord('q'):
          break  
  cap.release()
  cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
